In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


# Install Dependencies

In [1]:
!pip install transformers
!pip install accelerate
!pip install peft
!pip install datasets
!pip install wandb
!pip install bitsandbytes
!pip install huggingface-hub
!pip install einops
!pip install pandas
!pip install numpy

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-xr0ckmrt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-xr0ckmrt
  Resolved https://github.com/huggingface/transformers to commit 9dc4ce9ea73dd46c4f63a3182a8cbd1cef1b886f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-2ljlei1z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-2ljlei1z
  Resolved https://github.com/huggingface/peft to commit 884b1ac3a8ef49c9301b5bbf02e8bc64349e95f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Add credentials

In [ ]:
#!wandb login --relogin

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Download base model and dataset

In [2]:
from datasets import load_dataset
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
import torch

from transformers import BitsAndBytesConfig

quantization_config = None


model_name = "bigcode/starcoderbase-1b"
dataset_name = "flytech/llama-python-codes-30k"
dataset_split = "train[:5000]"

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset(dataset_name,split=dataset_split)


ImportError: ignored

In [4]:
print(model.get_memory_footprint())

4615938048


# Prepare the dataset

In [ ]:
def remove_llama_tokens(example):
  remove_tokens = ['[INST]','[/INST]','<s>','</s>', '```python', '```']
  for token in remove_tokens:
    example['instruction'] = example['instruction'].replace(token,'').strip()
    example['output'] = example['output'].replace(token,'').strip()
  example['text'] = '# '+example['instruction'] + '\n\n' + example['output']
  return example

def convert_to_pt(example):
  example['input_ids'] = torch.tensor(example['input_ids'])
  example['attention_mask'] = torch.tensor(example['attention_mask'])
  return example



def tokenize_function(examples, text_column='text'):
    return tokenizer(examples[text_column], return_tensors="pt",padding=True)


def prepare_datasets(tokenizer, dataset, test_size=.2):
  dataset = dataset.map(remove_llama_tokens)
  print(f"Tokenizing dataset ...")
  tokenized_dataset = dataset.map(tokenize_function, batched=True)



  return tokenized_dataset

In [ ]:
dataset = dataset.map(remove_llama_tokens)


In [ ]:
max_length = max([len(tokenizer(x['text'])["input_ids"]) for x in dataset])
print(max_length)

423


In [ ]:

def preprocess_function(examples, text_column='text'):
    batch_size = len(examples[text_column])
    inputs = [str(x) for x in examples[text_column]]
    targets = [str(x) for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset.column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
processed_datasets = processed_datasets.train_test_split(test_size=0.3)

# Load Model

In [ ]:
from peft import LoraConfig, get_peft_model


peft_config = LoraConfig(
  lora_alpha=32,
  lora_dropout=.1,
  r=4,
  bias="none",
  task_type="CAUSAL_LM",
  target_modules="c_proj,lm_head".split(","),
)



model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,581,056 || all params: 1,138,788,352 || trainable%: 0.13883668525615547


In [ ]:
 # TODO: more LoRA Modules?

# Prepare Training!

In [ ]:
from tqdm import tqdm
device = "cuda"
num_epochs = 5
lr=5e-5
batch_size = 64
iter_batch_size=4

log_interval = batch_size // iter_batch_size

In [ ]:
train_dataloader = DataLoader(
    processed_datasets['train'], shuffle=True, collate_fn=default_data_collator, batch_size=iter_batch_size, pin_memory=True
)

eval_dataloader = DataLoader(
    processed_datasets['test'], shuffle=True, collate_fn=default_data_collator, batch_size=iter_batch_size, pin_memory=True
)

In [ ]:


# model
# optimizer and lr scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)



In [ ]:
# training and evaluation
model = model.to(device);


In [ ]:
vibe_check_input = tokenizer("# write a basic cnn in pytorch", return_tensors="pt")
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in vibe_check_input.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=256, eos_token_id=0
    )
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


tensor([[   21, 11638,   667,   372,  1930,  3654,   688,   322,  2297,   438,
          3892,   203,   589, 11638,    81,  1126,  2262,   284,  3097,   203,
           203,    21, 11638,   667,   372,  1930,  3654,   688,   322,  2297,
           438,  3892,   203,   589, 11638,    81,  1126,    81,    36,  2262,
           284,  3097,   203,   203,    21, 11638,   667,   372,  1930,  3654,
           688,   322,  2297,   438,  3892,   203,   589, 11638,    81,  1126,
            81,    37,  2262,   284,  3097,   203,   203,    21, 11638,   667,
           372,  1930,  3654,   688,   322,  2297,   438,  3892,   203,   589,
         11638,    81,  1126,    81,    38,  2262,   284,  3097,   203,   203,
            21, 11638,   667,   372,  1930,  3654,   688,   322,  2297,   438,
          3892,   203,   589, 11638,    81,  1126,    81,    39,  2262,   284,
          3097,   203,   203,    21, 11638,   667,   372,  1930,  3654,   688,
           322,  2297,   438,  3892,   203,   589, 1

In [ ]:

# Next steps:

best_loss = 1e9

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        if step % log_interval == 0:
          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()

    eval_epoch_loss = eval_loss / len(eval_dataloader)

    eval_epoch_loss = eval_epoch_loss.cpu().item()

    if eval_epoch_loss < best_loss:
      print('saving model to hub...')
      best_loss = eval_epoch_loss
      # save best model to HF Hub

    train_epoch_loss = total_loss / len(train_dataloader)
    train_epoch_loss = train_epoch_loss.cpu().item()

    print(f"{epoch=}: {train_epoch_loss=}  {eval_epoch_loss=}")

100%|██████████| 750/750 [07:33<00:00,  1.65it/s]


epoch=0: train_ppl=tensor(1.4582, device='cuda:0') train_epoch_loss=tensor(0.3772, device='cuda:0') eval_ppl=tensor(1.4115, device='cuda:0') eval_epoch_loss=tensor(0.3447, device='cuda:0')


100%|██████████| 750/750 [07:32<00:00,  1.66it/s]


epoch=1: train_ppl=tensor(1.3695, device='cuda:0') train_epoch_loss=tensor(0.3145, device='cuda:0') eval_ppl=tensor(1.3158, device='cuda:0') eval_epoch_loss=tensor(0.2745, device='cuda:0')


 94%|█████████▍| 1647/1750 [33:57<02:07,  1.24s/it]

In [ ]:
loss.detach().float()

tensor(0.1272, device='cuda:0')

In [ ]:
vibe_check_input = tokenizer("# plot bar graph of pandas df", return_tensors="pt")
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in vibe_check_input.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=256, eos_token_id=0    )
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


tensor([[   21,  6867,  5697,  3996,   432, 12643,  5018,   203,   203,   465,
         12500,    32, 17449,   619,  5743,   203,   465, 12643,   619,  6121,
           203,   203,  1479,   280,  6121,    32, 11364, 14178,    83,   950,
           428,    35,    30,   225,    36,    30,   225,    37,   614,   330,
            84,   950,   428,    38,    30,   225,    39,    30,   225,    40,
         33993,   203,   203,  1479,    32,  3399,    32,  1632,   346,   203,
           203,    21,  6867,  5697,  3996,   432,  6436,  1417,   203,   203,
           465, 12500,    32, 17449,   619,  5743,   203,   465,  6436,   619,
          2065,   203,   203,   106,   280,  2065,    32, 16251,    26,    35,
            30,   225,    41,    27,   203,   203,   107,   280,  2065,    32,
          3855,    32, 39385,    26,    41,    27,   203,   203,  6073,    32,
          1632,    26,   106,    30,   533,    27,   203,   203,    21,  6867,
          5697,  3996,   432, 12643,  5018,   203,  

# Load this model and adapter in the future

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


peft_model_id = "nkasmanoff/tmp_trainer"
model_name = "bigcode/starcoderbase-1b"
model = AutoModelForCausalLM.from_pretrained(model_name)
model.load_adapter(peft_model_id)

In [ ]:
model.load_adapter?